# Adversarial Attacks

In [2]:
import tensorflow as tf
from keras.models import load_model

from data.file import model_convolutional_dataset
from data.load import load_train_dataset, load_test_dataset

In [3]:
model = load_model(model_convolutional_dataset)

In [ ]:
batch_size = 20
x_train, y_train = load_train_dataset()
x_test, y_test = load_test_dataset()

train_input = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_input = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
from reporting.images import display_dataframe
from keras.backend import argmax
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
import numpy as np

# epsilon: input variation parameter
eps = 10
count = 3
test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
test_acc_pgd = tf.metrics.SparseCategoricalAccuracy()

real = []
attacks = []
for x, y in train_input.take(count):
    x_fgm = fast_gradient_method(model, x, eps, np.inf)
    y_pred_fgm = model.predict(x_fgm)
    print("| real:", y.numpy(), " | prediction (pgd): ", argmax(y_pred_fgm).numpy(), " | accuracy:", test_acc_fgsm(y, y_pred_fgm).numpy().round(decimals=3), " |")

    real.extend(x.numpy())
    attacks.extend(x_fgm.numpy())

In [ ]:
display_dataframe(real, cols=10)

In [ ]:
display_dataframe(attacks, cols=10)